Liquid Time-Constant (LTC) networks are a continuous-time variant of recurrent neural nets. Each hidden unit is modeled as a dynamical system whose state evolves according to a first-order differential equation rather than a simple discrete update:

Hidden state x changes with a rate dx/dt that depends on the current input, the unit’s previous activation, a learned weight matrix θ, and a learnable “conductance” term A.
The time constant (the factor that says how quickly the state can change) is also learnable, so the network can adapt to different temporal scales—hence “liquid time-constant”.
This makes LTCs more flexible than standard RNNs or LSTMs when dealing with irregular time steps, missing samples, or long gaps in sequences: the dynamics fill in what happens between observations rather than forcing everything onto a fixed clock.

In the LTC tutorial notebook:

Hidden layer state x₁ is updated by integrating a simple ODE driven by the input I₀ and the previous state x₀, using parameters θ, A, and τ (the time constant).
Forward pass computes the updated hidden state and routes it to the output layer as y' = w·x₁ + b.
Training uses mean squared error and a straightforward gradient update to fit the toy target function y = 2x + 1.
Because the example keeps the “liquid” dynamics extremely simple (scalar parameters and one integration step per sample), it’s a didactic stand-in for the more complex continuous-time solvers used in full LTC models.
If you want to see the liquid behavior more clearly, try: (1) letting forward_pass integrate over multiple small time steps, (2) allowing the time constant to change during training, or (3) training on data with uneven sampling.

# Liquid Time-Constant Neural Network (LTC NN) Implementation
*Implemented by Michael B.Khani*
## Introduction
Liquid Time-Constant Neural Networks (LTC NN) are a type of neural network designed to learn complex functions. Their primary characteristic is the unique processing in the hidden layer which relies on prior activations.

In this tutorial, we'll implement an LTC NN to approximate the linear function y=2x+1. Our network will comprise of three layers:
1.   Input layer
2.   Hidden layer (with unique LTC NN processing)
3.   Output layer




## Step 1: Problem Definition
We aim to approximate the function y=2x+1 using a set of input-output pairs as our training data. The goal is to fine-tune the network's parameters such that its predictions are as close as possible to the true outputs.



## Step 2: Data Preparation
To approximate our target function, we'll generate training data for the first five integers (0 to 4):

In [7]:
#training_data = [(i, 2*i + 1) for i in range(5)]
training_data = [(0, 1), (1, 3), (2, 5), (3, 7), (4, 9)]

## Step 3: Network Architecture
1. Input Layer:
*   Single node for the input value I_0.
2. Hidden Layer:
*   Contains two nodes:
First node (x_0) always set to 1.0, acting as a bias.
Second node (x_1), processing input combined with the prior activation (x_0) using weights in theta and a constant A.
3. Output Layer:
*   A single node that produces the network's output. It computes the result using activations from the hidden layer combined with weight w and bias b.

## Step 4: Initialization
Before training, we must initialize the network parameters:

In [8]:
import numpy as np # NumPy is a fundamental package for scientific computing with Python

theta = np.array([0.5, -0.2])
A = np.array([0.3])
tau = 1.0
w = 0.4
b = -0.1
learning_rate = 0.1


## Step 5: Forward Propagation
The `forward_pass` function is used to compute the network's output for a given input. It processes data through the layers, applying appropriate weights and biases.

In [9]:
def forward_pass(input_value):
    I_0 = input_value
    x_0 = 1.0  # Use 1.0 instead of 0.0 to avoid vanishing activations initially
    S_0 = I_0 * theta[0] * A * x_0 + x_0 * theta[1] * A * x_0
    dx_dt_0 = x_0/tau + S_0
    x_1 = x_0 + dx_dt_0
    y_prime = x_1 * w + b
    return y_prime

Example for input_value = 2 using the parameters in LTC_Instructions.ipynb (theta = [0.5, -0.2], A = 0.3, tau = 1.0, w = 0.4, b = -0.1):

I_0 = 2, x_0 = 1.0
S_0 = I_0*theta[0]*A*x_0 + x_0*theta[1]*A*x_0 = (2*0.5*0.3) + (1*-0.2*0.3) = 0.24
dx_dt_0 = x_0/tau + S_0 = 1.0 + 0.24 = 1.24
x_1 = x_0 + dx_dt_0 = 1.0 + 1.24 = 2.24
y_prime = x_1*w + b = 2.24*0.4 - 0.1 = 0.796
If you keep A as the 1-element NumPy array (np.array([0.3])), the code returns the same value wrapped in an array: [0.796]

theta = [0.5, -0.2] is just an initialization choice for the two learned coefficients in that toy liquid neuron. The positive 0.5 scales how strongly the current input drives the state, while the negative −0.2 gives a bit of inhibitory/self-damping influence from the existing activation. Picking one positive and one negative value keeps the demonstration interesting (the neuron doesn’t explode or collapse to zero immediately), and the training loop is expected to update them from those starting points. You’re free to pick any other reasonable pair; they’re not derived constants.

That second term models self-interaction of the current state. In the simplified liquid neuron recipe in LTC_Instructions.ipynb, S_0 is built from two bilinear parts: one coupling the input with the state (I_0 * theta[0] * A * x_0), and one coupling the state with itself (x_0 * theta[1] * A * x_0). Because the state value x_0 appears in both factors of that second product, it collapses to theta[1] * A * x_0^2. It’s just a compact way to say “scale the recurrent feedback by both the current activation and its own influence,” giving you a quadratic (self-damping or self-excitatory) term controlled by theta[1].

## Step 6: Loss Calculation
We measure how close the network's predictions are to the true outputs using the Mean Squared Error (MSE) loss.



In [10]:
def calculate_loss(y_prime, desired_output):
    return (y_prime - desired_output) ** 2

## Step 7: Backward Propagation
To optimize the network, we need to adjust its parameters based on the error it makes. The `backward_pass` function computes the gradients required to adjust the weights and biases.

In [11]:
def backward_pass(y_prime, desired_output, input_value, x_t):
    dL_dy_prime = 2 * (y_prime - desired_output)
    dL_dw = dL_dy_prime * x_t
    dL_db = dL_dy_prime
    dL_dtheta_0 = dL_dy_prime * w * input_value
    dL_dtheta_1 = dL_dy_prime * w * x_t

    return dL_dw, dL_db, dL_dtheta_0, dL_dtheta_1

## Step 8: Accuracy Evaluation
Accuracy quantifies how well our model is performing. We want to know the percentage of predictions that are within a 5% margin of the true outputs:

In [12]:
def calculate_accuracy():
    accurate_data_points = 0
    for input_value, desired_output in training_data:
        y_prime = forward_pass(input_value)
        percentage_error = abs((y_prime - desired_output) / desired_output) * 100
        if percentage_error < 5:
            accurate_data_points += 1
    return (accurate_data_points / len(training_data)) * 100

## Step 9: Training the LTC NN
Now, we'll iteratively adjust our network's parameters using the training data until our model reaches an accuracy of over 95%:

In [13]:
# Training loop
while calculate_accuracy() < 95:
    total_loss = 0
    for input_value, desired_output in training_data:
        y_prime = forward_pass(input_value)
        loss = calculate_loss(y_prime, desired_output)
        total_loss += loss

        dL_dw, dL_db, dL_dtheta_0, dL_dtheta_1 = backward_pass(y_prime, desired_output, input_value, 1.0)  # x_t=1.0

        w -= learning_rate * dL_dw
        b -= learning_rate * dL_db
        theta[0] -= learning_rate * dL_dtheta_0
        theta[1] -= learning_rate * dL_dtheta_1

    print(f"Loss = {total_loss}, Accuracy = {calculate_accuracy()}%")

Loss = [12.78858314], Accuracy = 0.0%
Loss = [13.75772574], Accuracy = 20.0%
Loss = [10.34141747], Accuracy = 20.0%
Loss = [6.91020373], Accuracy = 20.0%
Loss = [4.72203731], Accuracy = 20.0%
Loss = [3.38628261], Accuracy = 20.0%
Loss = [2.53838222], Accuracy = 20.0%
Loss = [1.97054546], Accuracy = 0.0%
Loss = [1.57153825], Accuracy = 0.0%
Loss = [1.27979247], Accuracy = 0.0%
Loss = [1.05945549], Accuracy = 0.0%
Loss = [0.8885917], Accuracy = 0.0%
Loss = [0.75317769], Accuracy = 0.0%
Loss = [0.64390038], Accuracy = 0.0%
Loss = [0.55436818], Accuracy = 0.0%
Loss = [0.48006781], Accuracy = 0.0%
Loss = [0.41773239], Accuracy = 0.0%
Loss = [0.36494519], Accuracy = 0.0%
Loss = [0.31988387], Accuracy = 0.0%
Loss = [0.28115062], Accuracy = 0.0%
Loss = [0.24765696], Accuracy = 0.0%
Loss = [0.21854358], Accuracy = 0.0%
Loss = [0.19312381], Accuracy = 0.0%
Loss = [0.17084275], Accuracy = 20.0%
Loss = [0.15124751], Accuracy = 20.0%
Loss = [0.133965], Accuracy = 20.0%
Loss = [0.11868528], Accuracy

/tmp/ipykernel_4957/838649029.py:13: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  theta[0] -= learning_rate * dL_dtheta_0
/tmp/ipykernel_4957/838649029.py:14: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  theta[1] -= learning_rate * dL_dtheta_1


## Step 10: Testing the LTC NN
Finally, once the network is trained, we can use it to make predictions and compare them to the actual outputs:



In [14]:
# Test
print(f"\nTraining Data: {training_data}")
for input_value, _ in training_data:
    print(f"Input: {input_value}, Predicted Output: {forward_pass(input_value)}, Actual Output: {2*(input_value)+1}")


Training Data: [(0, 1), (1, 3), (2, 5), (3, 7), (4, 9)]
Input: 0, Predicted Output: [1.04937744], Actual Output: 1
Input: 1, Predicted Output: [3.06861444], Actual Output: 3
Input: 2, Predicted Output: [5.08785144], Actual Output: 5
Input: 3, Predicted Output: [7.10708844], Actual Output: 7
Input: 4, Predicted Output: [9.12632544], Actual Output: 9


## Conclusion
You've now implemented a simple Liquid Time-Constant Neural Network (LTC NN) from scratch and trained it to approximate a linear function. While this example is elementary, the principles applied here can be expanded for more complex tasks.


---
> Note: Liquid Time-constant Neural networks (LTC NNs) are an interesting architecture that has the advantage of being relatively insensitive to gap length, compared to other RNNs, hidden Markov models, and other sequence learning methods1. While LTC NNs may not be as common as other architectures like CNNs or RNNs, it is always important to consider the nature and requirements of the problem at hand before choosing a suitable neural network architecture.
---


To get a full understanding, run the provided Python code and analyze the outputs. Experiment by changing parameters, training data, or network structure to gain deeper insights.